In [1]:
import pandas as pd
from lxml import etree
import Capstone_functions as cf
import os
import sql_functions as sf

#### Global variables

In [6]:
parser = etree.XMLParser(recover=True)
engine = sf.get_engine()
schema = "BGG_Data"
xtree = etree.parse("data/old_API_id_260200.xml", parser= parser).getroot()
subnodes = [
    'boardgamecategory',
    'boardgamesubdomain',
    'boardgamemechanic',
    'boardgamefamily',
    'boardgameexpansion',
    'boardgamehonor',
    'boardgamedesigner',
    'boardgameartist',
    'boardgamepublisher',
    'boardgamepodcastepisode',
    'boardgameimplementation',
    'videogamebg',
    'statistics',
    'marketplacelistings'
]

#### Build df_main

In [7]:
df_main = pd.DataFrame()
directory = os.fsencode("data/")

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xml"): 
        file_path = os.path.join(str(directory)[2:].replace("'",""), filename)
        xtree_temp = etree.parse(file_path, parser= parser).getroot()
        df_main = pd.concat([df_main,cf.df_main(xtree_temp)])


In [8]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333317 entries, 0 to 185
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             333317 non-null  object
 1   description    333317 non-null  object
 2   yearpublished  137153 non-null  object
 3   min_players    137153 non-null  object
 4   max_players    137153 non-null  object
 5   playtime       137153 non-null  object
 6   min_playtime   137153 non-null  object
 7   max_playtime   137153 non-null  object
 8   min_age        137153 non-null  object
dtypes: object(9)
memory usage: 25.4+ MB


#### Loop over subnodes and add dataframes to dictionary

In [9]:
df_dict = dict()
directory = os.fsencode("data/")
for entrypoint in subnodes:
    df_temp = pd.DataFrame()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".xml"): 
            file_path = os.path.join(str(directory)[2:].replace("'",""), filename)
            xtree_temp = etree.parse(file_path, parser= parser).getroot()
            df_temp = pd.concat([df_temp,cf.df_subnodes(xtree_temp, entrypoint=entrypoint)])
    df_dict[entrypoint] = df_temp

#### Show info() for all the subnode-dataframes

In [ ]:
for node in subnodes:
    print(node)
    df_dict[node].info()

#### upload df_main to the postgres database

In [ ]:
sf.build_table(engine=engine,table_name="df_main_dirty",dataframe=df_main,schema=schema)

#### upload all the subnode dataframes ('boardgamecategory',........,'marketplacelistings') to the postgres-server

In [ ]:
for entrypoint in subnodes:
    sf.build_table(engine=engine,table_name=entrypoint,dataframe=df_dict[entrypoint],schema=schema)

In [3]:
schema = "bgg_data"
table = "artist"

query = f"SELECT * FROM {schema}.{table}"

df = sf.get_dataframe(query)
df

,id,cat_id,cat_name
0,98401,42353,Sean Howard
1,98402,40734,Peke (ぺけ)
2,98416,26845,Francis Bretaudeau
3,98430,3,(Uncredited)
4,98436,3,(Uncredited)
...,...,...,...
98385,298175,28060,Lar DeSouza
98386,298191,123860,Francisco de la Riva Agüero
98387,298195,77084,Alexandr Elichev
98388,298195,78961,Josh T. McDowell
